In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


In [ ]:
import io
df = pd.read_csv('/home/vikasnr/codebase/crsl/tuesday/llm-cls/method_1/llm_selection_data_500.csv')
df.drop(columns=["domain"],inplace=True)
df.LLM.value_counts()

LLM
GPT        186
Claude     170
Mistral    144
Name: count, dtype: int64

In [27]:
df.head()


,task_type,input_token_length,output_token_length,multimodal,structured_input,numerical_reasoning,multi_turn_conversation,grounding_required,creativity_level,response_factuality,...,context_window,memory_usage_GB,cost_per_1K_tokens,on_prem_support,cloud_provider,gpu_availability,fine_tuning_available,compliance_requirements,bias_score,LLM
0,Code Generation,46,202,0,1,0,0,1,5,2,...,4096,33.6,0.01656,0,Azure,1,1,HIPAA,0.595,Claude
1,Code Generation,904,1094,0,0,0,1,0,3,1,...,16384,34.5,0.00895,1,OpenAI,0,1,GDPR,0.933,Mistral
2,Classification,560,344,1,0,1,1,1,1,5,...,16384,23.5,0.01584,0,AWS,1,1,HIPAA,0.446,GPT
3,Classification,349,568,0,0,0,1,1,1,3,...,16384,26.6,0.00659,1,GCP,0,0,GDPR,0.954,Claude
4,Summarization,261,1757,1,1,1,1,1,2,1,...,16384,34.3,0.01144,1,OpenAI,1,1,NaN,0.558,Mistral


In [ ]:
# !pip install -U imbalanced-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


# Identify categorical and numerical columns
# categorical_cols = ["task_type", "domain", "cloud_provider", "compliance_requirements"]
categorical_cols = ["task_type", "cloud_provider", "compliance_requirements"]

numerical_cols = [col for col in df.columns if col not in categorical_cols + ["LLM"]]

# Encode categorical variables
label_encoders = {col: LabelEncoder() for col in categorical_cols}
for col in categorical_cols:
    df[col] = label_encoders[col].fit_transform(df[col])

# Encode target variable
target_encoder = LabelEncoder()
df["LLM"] = target_encoder.fit_transform(df["LLM"])

# Separate target variable (LLM) and features
X = df.drop(columns=["LLM"])
y = df["LLM"]

# Scale numerical features
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train an XGBoost classifier
clf = XGBClassifier(n_estimators=50, max_depth=3, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=target_encoder.classes_)

print(f"Accuracy: {accuracy}\n")
print(report)


Accuracy: 0.44642857142857145

              precision    recall  f1-score   support

      Claude       0.24      0.30      0.26        30
         GPT       0.48      0.42      0.45        38
     Mistral       0.61      0.57      0.59        44

    accuracy                           0.45       112
   macro avg       0.44      0.43      0.43       112
weighted avg       0.47      0.45      0.45       112



/home/vikasnr/miniconda3/envs/mispix_clone/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [11:27:21] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
# Based on the nature of the data and the goal of predicting the 'LLM' (Large Language Model), some features might be considered less impactful or potentially redundant. Here's a breakdown of features that could be considered for ignoring or further analysis:

# Potentially Ignorable or Less Impactful Features:

# input_token_length and output_token_length:
# While these features provide information about the size of the input and output, their direct correlation with the specific LLM might be complex. LLM selection is often driven by capabilities, cost, and compliance more than raw token counts. However, they could be useful for performance analysis, not direct LLM prediction.
# latency_ms and token_throughput:
# These are performance metrics that can vary significantly based on hardware, network conditions, and specific usage patterns. They might be more reflective of the deployment environment than the inherent characteristics of the LLM. Again, usefull for performance analysis.
# memory_usage_GB:
# Similar to latency and throughput, memory usage is highly dependent on the deployment environment and the specific task.
# BLEU_score, ROUGE_score, accuracy_score, faithfulness_score, hallucination_rate, bias_score:
# These are evaluation metrics that reflect the performance of the LLM on specific tasks. They are results of using the LLM, not intrinsic properties that determine which LLM is chosen. Because of this, they are usefull to evaluate the model, but not to predict which model was used.
# context_window:
# While context window size is a characteristic of an LLM, it might be highly correlated with other features (like memory_usage_GB or gpu_availability). If other features already capture the essence of computational requirements, this might be redundant.
# Reasons for Potential Redundancy:

# Deployment Dependence: Many performance-related metrics (latency, throughput, memory) are influenced by factors outside the LLM itself.   
# Resultant Metrics: Evaluation scores are the outcome of using the LLM, not its defining features.
# Correlation: Some features might be highly correlated, providing redundant information.   
# Important Considerations:

# Domain Expertise: The significance of features can vary depending on the specific domain. If the analysis is focused on performance optimization, then latency and throughput become crucial.
# Feature Importance Analysis: Techniques like feature importance from tree-based models (e.g., Random Forest) or permutation importance can help quantify the contribution of each feature to the model's predictions.   
# Recommendation:

# Start by training a model with all features.
# Then, perform feature importance analysis to identify the most influential features.
# Experiment with removing the potentially redundant or less impactful features and observe the model's performance.
# By carefully considering these factors, you can refine your feature set and build a more robust and efficient model.

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

import io
df = pd.read_csv('/home/vikasnr/codebase/crsl/tuesday/llm-cls/method_1/llm_selection_data_500.csv')
df.drop(columns=["domain"],inplace=True)
df.head()
df.LLM.value_counts()
data = df
# import io
# data = pd.read_csv(io.StringIO(data))

# Separate features and target
X = data.drop('LLM', axis=1)
y = data['LLM']

# Drop potentially less impactful features
X = X.drop([
    'input_token_length', 'output_token_length', 'latency_ms', 'token_throughput',
    'memory_usage_GB', 'BLEU_score', 'ROUGE_score', 'accuracy_score',
    'faithfulness_score', 'hallucination_rate', 'bias_score', 'context_window'
], axis=1, errors='ignore')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42)) # Using RandomForestClassifier
])

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Best Parameters: {'classifier__max_depth': None, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 100}
Accuracy: 0.33
Classification Report:
               precision    recall  f1-score   support

      Claude       0.33      0.21      0.26        38
         GPT       0.34      0.54      0.42        39
     Mistral       0.29      0.17      0.22        23

    accuracy                           0.33       100
   macro avg       0.32      0.31      0.30       100
weighted avg       0.32      0.33      0.31       100



In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import io
df = pd.read_csv('/home/vikasnr/codebase/crsl/tuesday/llm-cls/method_1/llm_selection_data_500.csv')
df.drop(columns=["domain"],inplace=True)
df.head()
df.LLM.value_counts()

data = df

# Separate features and target
X = data.drop('LLM', axis=1)
y = data['LLM']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numerical and categorical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create preprocessing pipeline
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GaussianNB())
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=0)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.31
Classification Report:
               precision    recall  f1-score   support

      Claude       0.36      0.34      0.35        38
         GPT       0.35      0.38      0.37        39
     Mistral       0.14      0.13      0.14        23

    accuracy                           0.31       100
   macro avg       0.28      0.29      0.28       100
weighted avg       0.31      0.31      0.31       100

